In [1]:
import ROOT
from ROOT import gStyle
import numpy as np

def biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def normalization(hist_list):
    for hist in hist_list:
        for i in range(0,len(rebining)):
            value=hist.GetBinContent(i)
            sf=hist.GetBinWidth(i)/5.0    
            hist.SetBinContent(i,value/sf)

Welcome to JupyROOT 6.18/04


In [2]:
#CALCULATION OF RQCD

rebin="n"
histo="delta_phi_cuts"

MC_CR_OS=ROOT.TFile.Open("CR_OS/MC.root","READ")
Data_CR_OS = ROOT.TFile.Open("CR_OS/Data.root","READ")
MC_CR_SS=ROOT.TFile.Open("CR_SS/MC.root","READ")
Data_CR_SS = ROOT.TFile.Open("CR_SS/Data.root","READ")



a=histo

mc_cr_os=MC_CR_OS.Get(histo)
data_cr_os=Data_CR_OS.Get(histo)
mc_cr_ss=MC_CR_SS.Get(histo)
data_cr_ss=Data_CR_SS.Get(histo)

mc_cr_os.SetDirectory(0)
data_cr_os.SetDirectory(0)
mc_cr_ss.SetDirectory(0)
data_cr_ss.SetDirectory(0)

MC_CR_OS.Close()
Data_CR_OS.Close()
MC_CR_SS.Close()
Data_CR_SS.Close()

data_cr_os.Add(mc_cr_os,-1)
data_cr_ss.Add(mc_cr_ss,-1)

RQCD=data_cr_os.Integral()/data_cr_ss.Integral()

# HISTOGRAM FOR SR_OS

MC_SR_OS=ROOT.TFile.Open("SR_OS/MC.root","READ")
Data_SR_OS = ROOT.TFile.Open("SR_OS/Data.root","READ")
MC_SR_SS=ROOT.TFile.Open("SR_SS/MC.root","READ")
Data_SR_SS = ROOT.TFile.Open("SR_SS/Data.root","READ")

mc_sr_os=MC_SR_OS.Get(histo)
data_sr_os=Data_SR_OS.Get(histo)
mc_sr_ss=MC_SR_SS.Get(histo)
data_sr_ss=Data_SR_SS.Get(histo)

mc_sr_os.SetDirectory(0)
data_sr_os.SetDirectory(0)
mc_sr_ss.SetDirectory(0)
data_sr_ss.SetDirectory(0)

MC_SR_OS.Close()
Data_SR_OS.Close()
MC_SR_SS.Close()
Data_SR_SS.Close()

ob=data_sr_ss.GetNbinsX()
s=data_sr_ss.GetXaxis().GetBinLowEdge(1)
e=data_sr_ss.GetXaxis().GetBinUpEdge(data_sr_ss.GetNbinsX())
n=5
if rebin=="y":
    rebining=biner(s,180,e,10,20)
    nb=len(rebining)-1
    mc_sr_ss=mc_sr_ss.Rebin(nb,"mc_sr_ss",rebining)
    data_sr_ss=data_sr_ss.Rebin(nb,"data_sr_ss",rebining)


MJ_BG=data_sr_ss.Clone()
MJ_BG.Add(mc_sr_ss,-1)
MJ_BG.Scale(RQCD)

In [3]:
file=ROOT.TFile.Open("SR_OS/MJ.root","update")
MJ_BG.Write(a)
file.Close()

In [4]:

Data = ROOT.TFile.Open("SR_OS/Data.root","READ")
Signal=ROOT.TFile.Open("SR_OS/Signal.root","READ")
VV=ROOT.TFile.Open("SR_OS/VV.root","READ")
Wjets=ROOT.TFile.Open("SR_OS/Wjets.root","READ")
Zjets=ROOT.TFile.Open("SR_OS/Zjets.root","READ")
Ttbar=ROOT.TFile.Open("SR_OS/ttbar.root","READ")
SingleTop=ROOT.TFile.Open("SR_OS/singletop.root","READ")
MJ=ROOT.TFile.Open("SR_OS/MJ.root","READ")

data=Data.Get(a)
signal=Signal.Get(a)
vv=VV.Get(a)
w=Wjets.Get(a)
z=Zjets.Get(a)
tt=Ttbar.Get(a)
t=SingleTop.Get(a)
mj=MJ.Get(a)

signal.SetDirectory(0)
data.SetDirectory(0)
vv.SetDirectory(0)
w.SetDirectory(0)
z.SetDirectory(0)
tt.SetDirectory(0)
t.SetDirectory(0)
mj.SetDirectory(0)

Signal.Close()
Data.Close()
VV.Close()
Wjets.Close()
Zjets.Close()
Ttbar.Close()
SingleTop.Close()
MJ.Close()

hs = ROOT.THStack("hs","")
if rebin=="y":
    signal=signal.Rebin(nb,"signal",rebining)
    data=data.Rebin(nb,"data",rebining)
    vv=vv.Rebin(nb,"vv",rebining)
    tt=tt.Rebin(nb,"tt",rebining)
    t=t.Rebin(nb,"t",rebining)
    z=z.Rebin(nb,"z",rebining)
    w=w.Rebin(nb,"w",rebining)

ratio=w.Clone()
lista = ROOT.TList()
lista.Add(mj)
lista.Add(z)
lista.Add(vv)
lista.Add(tt)
lista.Add(t)
lista.Add(signal)
ratio.Merge(lista)
ratio.Divide(data)



if rebin=="y":
    hist_list=[signal,data,vv,tt,t,z,w,mj]
    normalization(hist_list)

w.SetFillColor(ROOT.kGreen)
w.SetLineColor(ROOT.kGreen)
z.SetFillColor(ROOT.kRed)
z.SetLineColor(ROOT.kRed)
vv.SetFillColor(ROOT.kBlue)
vv.SetLineColor(ROOT.kBlue)
tt.SetFillColor(ROOT.kYellow)
tt.SetLineColor(ROOT.kYellow)
t.SetFillColor(ROOT.kCyan)
t.SetLineColor(ROOT.kCyan)
mj.SetLineColor(ROOT.kMagenta)
mj.SetFillColor(ROOT.kMagenta)
data.SetLineColor(ROOT.kBlack)



hs.Add(vv)
hs.Add(w)
hs.Add(z)
hs.Add(tt)
hs.Add(t)
hs.Add(mj)
hs.Add(signal)




gStyle.SetOptStat(1111111)

canvas3 = ROOT.TCanvas("canvas3")
canvas3.cd()

pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)

pad1.Draw ()
pad1.cd ()

gStyle.SetStatY(0.9);                
gStyle.SetStatX(0.9);
gStyle.SetStatW(0.1);                
gStyle.SetStatH(0.1); 
pad1.SetLogy(0)


data.Draw("pe")
hs.Draw("HIST same")
data.Draw("pe same")
data.GetXaxis().Draw()

pad1.RedrawAxis()

pad1.SetTicks(1,1)
pad1.Update()
data . GetYaxis (). SetRangeUser (1 ,1.5*data.GetBinContent(data.GetMaximumBin()))
data . GetXaxis (). SetRangeUser (s ,e)
data . GetYaxis (). SetTitle ("Events/0.1")
#data . GetXaxis (). SetTitle ("GeV")

hs . GetXaxis (). SetRangeUser (s ,e)
legend = ROOT . TLegend (0.7 ,0.68 ,0.8 ,0.88)
legend . AddEntry ( vv ,"Diboson")
legend . AddEntry ( data ," Data ")
legend . AddEntry ( signal ," Signal ")
legend . AddEntry ( w ," W+jets ")
legend . AddEntry ( z ," Z+jets ")
legend . AddEntry ( t ," Single top ")
legend . AddEntry ( tt ," ttbar ")
legend . AddEntry ( mj,"MJ")

legend . SetLineWidth (0)
legend . Draw ()



hs.SetTitle(a)

canvas3.cd()
pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.35)
pad2.Draw ()
pad2.cd ()
ratio.Draw ("pe")
ratio.SetStats(0)
ratio . GetYaxis (). SetRangeUser (0 ,2.0)
ratio . GetXaxis (). SetRangeUser (s ,e)
ratio . GetYaxis (). SetTitle ("MC/DATA")
ratio . GetYaxis (). SetTitleSize (0.15)
ratio . GetYaxis (). SetTitleOffset (0.25)
line = ROOT . TLine (s ,1 ,e,1)
line . SetLineColor ( ROOT . kBlack )
line . SetLineWidth (2)
line . Draw (" same ")


canvas3.Print(a+".pdf")


Info in <TCanvas::Print>: pdf file delta_phi_cuts.pdf has been created
